데이터가 Sparse matrix 일 때 cosine distance 를 이용하면 대부분의 거리가 0.9 ~ 1.0 에 가깝습니다. 그렇기 때문에 bag of words model 을 이용하는 term frequency matrix 에서는 k-means++ 가 큰 의미가 없습니다. 

k-means++ 은 distance distribution 이 한쪽에 치우치지 않았을 때 잘 작동합니다. Similar cut initializer 는 term frequency matrix 와 같은 sparse vector 에서 효율적으로 initial points 를 선택하기 위한 방법입니다. 더 자세한 설명은 아래의 블로그를 참고해주세요.

https://lovit.github.io/nlp/machine%20learning/2018/03/19/kmeans_initializer/

실험에 이용하는 데이터는 https://github.com/lovit/textmining_dataset/ 에서 다운받을 수 있습니다.

In [1]:
from lovit_textmining_dataset.navernews_10days import get_bow

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
x.shape

(30091, 9774)

idx_to_vocab 은 9.774 개의 단어들의 idx 에 해당하는 단어가 저장되어 있습니다. 이는 cluster labeling 에 이용됩니다.

In [2]:
print(idx_to_vocab[5535:5540])

['아이스크림', '아이엠', '아이오아이', '아이콘', '아이템']


In [3]:
import time
import sys
sys.path.append('../')
import soyclustering

print(soyclustering.__version__)

n_clusters = 300

0.2.0


## Time comparison

### Initializer: similar cut vs. k-means++

k-means++ 을 이용할 경우 (30091, 9774) 데이터에 대하여 initialization 에 9.462 초를 이용합니다.

similar cut 은 비슷한 점들을 initial points 로 선택하지 않으면서도 0.043 초만에 initialization 이 가능합니다. distance distribution 이 0.9 ~ 1.0 에 치우친 경우에는 k-means++ 은 잘 작동하지 않습니다.

In [4]:
import numpy as np
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='k-means++',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True,
    random_state=0
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=9.648999 sec, sparsity=0.00844
n_iter=1, changed=29952, inertia=16427.832, iter_time=1.990 sec, sparsity=0.164
n_iter=2, changed=5975, inertia=12413.602, iter_time=1.922 sec, sparsity=0.152
n_iter=3, changed=2192, inertia=11976.571, iter_time=1.924 sec, sparsity=0.148
n_iter=4, changed=1128, inertia=11838.584, iter_time=1.942 sec, sparsity=0.147
n_iter=5, changed=701, inertia=11764.806, iter_time=1.913 sec, sparsity=0.146
n_iter=6, changed=390, inertia=11726.887, iter_time=1.914 sec, sparsity=0.145
n_iter=7, changed=299, inertia=11710.406, iter_time=1.899 sec, sparsity=0.145
n_iter=8, changed=198, inertia=11700.876, iter_time=1.913 sec, sparsity=0.145
n_iter=9, changed=159, inertia=11694.908, iter_time=1.907 sec, sparsity=0.145
n_iter=10, changed=139, inertia=11689.538, iter_time=1.914 sec, sparsity=0.145
process time = 29.074 seconds


In [5]:
from soyclustering import SphericalKMeans

kmeans = SphericalKMeans(
    n_clusters=n_clusters,
    init='similar_cut',
    sparsity=None,
    max_iter=10,
    tol=0.0001,
    verbose = True,
    random_state=0
)

t = time.time()
labels = kmeans.fit_predict(x)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

initialization_time=0.049929 sec, sparsity=0.00707
n_iter=1, changed=29951, inertia=16428.435, iter_time=1.945 sec, sparsity=0.161
n_iter=2, changed=7805, inertia=12947.107, iter_time=1.916 sec, sparsity=0.149
n_iter=3, changed=3840, inertia=12239.101, iter_time=1.896 sec, sparsity=0.145
n_iter=4, changed=2191, inertia=11957.270, iter_time=1.927 sec, sparsity=0.142
n_iter=5, changed=1266, inertia=11826.583, iter_time=1.883 sec, sparsity=0.141
n_iter=6, changed=827, inertia=11762.833, iter_time=1.905 sec, sparsity=0.141
n_iter=7, changed=596, inertia=11717.253, iter_time=1.893 sec, sparsity=0.141
n_iter=8, changed=526, inertia=11689.117, iter_time=1.896 sec, sparsity=0.141
n_iter=9, changed=451, inertia=11667.402, iter_time=1.906 sec, sparsity=0.14
n_iter=10, changed=354, inertia=11647.316, iter_time=1.896 sec, sparsity=0.14
process time = 19.223 seconds


### soyclustering vs. scikit-learn

각 데이터와 centroids 와의 거리 계산 후, 새로운 cluster 에 데이터를 할당하는 부분이 scikit-learn 과 다르게 구현되어 있습니다. 현재 (version 0.2.0)는 scikit-learn 의 k-means 보다 약 1.9 배 정도 더 빨리 학습됩니다.

scikit-learn 은 Euclidean distance 를 이용합니다. L2 normalized 된 centroids 가 아닌 L1 normalized 이기 때문에 Cosine 과 Euclidean 의 값은 차이가 있습니다.

In [6]:
import sklearn
print(sklearn.__version__)

# scikit-learn KMeans use only Euclidean
from sklearn.preprocessing import normalize
x_ = normalize(x)

0.22.2.post1


In [7]:
from sklearn.cluster import KMeans

kmeans_sklearn = KMeans(
    n_clusters=n_clusters,
    init='k-means++',
    n_init=1,
    max_iter=10,
    tol=0.0001,
    verbose=1
)

t = time.time()
_label = kmeans_sklearn.fit_predict(x_)
t = time.time() - t

print('process time = {} seconds'.format('%.3f' % t))

Initialization complete
Iteration  0, inertia 21420.119
Iteration  1, inertia 18232.433
Iteration  2, inertia 17830.319
Iteration  3, inertia 17662.184
Iteration  4, inertia 17561.343
Iteration  5, inertia 17484.342
Iteration  6, inertia 17430.595
Iteration  7, inertia 17390.316
Iteration  8, inertia 17365.886
Iteration  9, inertia 17345.277
process time = 57.047 seconds


## cluster labeling

proportion keywords 를 이용하면 cluster centroids 와 vocabulary index 를 이용하여 각 클러스터 별 cluster label 을 찾을 수 있습니다.

각 cluster 별로 weight 가 높은 candidates_topk 개의 후보 중에서 proportion keyword score 가 높은 topk 개의 단어가 cluster labels 로 선택됩니다. 

Proportion keywords 의 원리는 아래의 블로그에 설명되어 있습니다.

https://lovit.github.io/nlp/machine%20learning/2018/03/21/kmeans_cluster_labeling/

In [8]:
from soyclustering import proportion_keywords

keywords = proportion_keywords(
    kmeans.cluster_centers_,
    labels,
    index2word=idx_to_vocab,
    topk=30,
    candidates_topk=100
)

/Users/hyunjoongkim/git/clustering4docs/soyclustering/_keyword.py:42: RuntimeWarning: invalid value encountered in true_divide
  def l1_normalize(x): return x/x.sum()
/Users/hyunjoongkim/git/clustering4docs/soyclustering/_keyword.py:67: RuntimeWarning: invalid value encountered in greater
  indices = np.where(p_prop > 0)[0]


keywords 는 list of list of tuple 입니다. Tuple 은 (단어, 키워드 점수)로 이뤄져 있습니다.

In [9]:
keywords[233]

[('빅브레인', 0.9998356991641689),
 ('너무너무너무', 0.999636565607772),
 ('신용재', 0.9996209421673745),
 ('오블리스', 0.9994778748084965),
 ('갓세븐', 0.9992970603778885),
 ('다비치', 0.9991180784713255),
 ('엠카운트다운', 0.9985666150928039),
 ('아이오아이', 0.9982645820718874),
 ('세븐', 0.996996851052826),
 ('박진영', 0.9967750269776107),
 ('방탄소년단', 0.9966473687693944),
 ('트로피', 0.996464966998091),
 ('완전체', 0.996227525576321),
 ('고마워', 0.995881757255629),
 ('산들', 0.9956056841004022),
 ('잠깐', 0.9955334197813145),
 ('중독성', 0.9949229577434185),
 ('펜타곤', 0.9946393121986635),
 ('정국', 0.9936714705659159),
 ('열창', 0.9934191181343699),
 ('그대', 0.9930229656843981),
 ('상큼', 0.9906486052581086),
 ('엠넷', 0.9896207484574001),
 ('타이틀곡', 0.9894943427370902),
 ('코드', 0.9890875408599619),
 ('챔피언', 0.9883417239438613),
 ('보컬', 0.9882745238500189),
 ('곡으로', 0.987120501433232),
 ('엑스', 0.9864101743091659),
 ('1위', 0.9845189730169617)]

특정 단어가 키워드에 포함된 cluster indices 를 저장하고, 해당 군집의 키워드를 출력합니다.

In [10]:
cluster_indices = []
for cluster_idx, keyword in enumerate(keywords):
    keyword = ' '.join([w for w,_ in keyword])
    if '아이오아이' in keyword:
        print('cluster#{} : {}'.format(cluster_idx, keyword))
        cluster_indices.append(cluster_idx)

cluster#233 : 빅브레인 너무너무너무 신용재 오블리스 갓세븐 다비치 엠카운트다운 아이오아이 세븐 박진영 방탄소년단 트로피 완전체 고마워 산들 잠깐 중독성 펜타곤 정국 열창 그대 상큼 엠넷 타이틀곡 코드 챔피언 보컬 곡으로 엑스 1위
cluster#298 : 다이아 에브리원 마법 고기 기희현 예능감 음악방송 한우 회식 댄스 대결 완전체 다리 아르바이트 3인 아이오아이 걸고 안무 플레이 6년 혼자 미스 출근 활동 타이틀곡 승리 활발 태양 아이돌 멤버들


특정 단어를 stopwords 에 추가하면 키워드를 추출할 때 해당 단어를 제외하고 키워드를 추출합니다.

In [11]:
keywords = proportion_keywords(
    kmeans.cluster_centers_,
    labels,
    index2word=idx_to_vocab,
    topk=30,
    candidates_topk=100,
    stopwords = {'아이오아이'}
)

In [12]:
for cluster_idx in cluster_indices:
    keyword = keywords[cluster_idx]
    keyword = ' '.join([w for w,_ in keyword])
    print('cluster#{} : {}'.format(cluster_idx, keyword))

cluster#233 : 빅브레인 너무너무너무 신용재 오블리스 갓세븐 다비치 엠카운트다운 세븐 박진영 방탄소년단 트로피 완전체 고마워 산들 잠깐 중독성 펜타곤 정국 열창 그대 상큼 엠넷 타이틀곡 코드 챔피언 보컬 곡으로 엑스 1위 눈물
cluster#298 : 다이아 에브리원 마법 고기 기희현 예능감 음악방송 한우 회식 댄스 대결 완전체 다리 아르바이트 3인 걸고 안무 플레이 6년 혼자 미스 출근 활동 타이틀곡 승리 활발 태양 아이돌 멤버들 먹고
